# 🌟 Chapter A4: Normal Maps & PBR

Phong lighting looks good, but AAA games use **Physically Based Rendering (PBR)**.

PBR simulates how light **actually** behaves in the real world, making materials look photorealistic.

## 1. Normal Mapping (Fake Geometry Detail)

A normal map stores surface direction in RGB channels:
- **R** = X direction
- **G** = Y direction
- **B** = Z direction

This lets us fake bumps and crevices without adding millions of triangles.

In [ ]:
from PIL import Image

# Load normal map (usually looks purplish/blue)
normal_map = Image.open('brick_normal.png')
normal_data = normal_map.transpose(Image.FLIP_TOP_BOTTOM).tobytes()

# Create texture
normal_tex = ctx.texture(normal_map.size, 3, normal_data)
normal_tex.build_mipmaps()

## 2. Tangent Space Calculations

Normal maps are in **tangent space** (local to the surface). We need to convert to world space.

In [ ]:
frag_shader_normal = '''
    #version 330
    in vec3 v_normal;
    in vec3 v_tangent;
    in vec3 v_bitangent;
    in vec2 v_uv;
    
    uniform sampler2D u_texture;
    uniform sampler2D u_normal_map;
    
    out vec4 f_color;
    
    void main() {
        // Sample normal map
        vec3 normal = texture(u_normal_map, v_uv).rgb;
        normal = normal * 2.0 - 1.0;  // Convert [0,1] to [-1,1]
        
        // Build TBN matrix (Tangent, Bitangent, Normal)
        mat3 TBN = mat3(v_tangent, v_bitangent, v_normal);
        normal = normalize(TBN * normal);
        
        // Use 'normal' for lighting calculations
        // ...
    }
'''

## 3. Introduction to PBR

PBR uses **physically accurate** material properties:

### Metallic Workflow
- **Albedo** (Base Color): The raw color without lighting
- **Metallic**: 0 = dielectric (plastic, wood), 1 = metal
- **Roughness**: 0 = mirror smooth, 1 = completely diffuse
- **Ambient Occlusion (AO)**: Shadows in crevices

In [ ]:
# Load PBR textures
albedo_tex = load_texture('metal_albedo.png')
metallic_tex = load_texture('metal_metallic.png')
roughness_tex = load_texture('metal_roughness.png')
ao_tex = load_texture('metal_ao.png')
normal_tex = load_texture('metal_normal.png')

## 4. PBR Fragment Shader (Simplified)

Full PBR is complex, but here's a simplified version:

In [ ]:
pbr_frag = '''
    #version 330
    in vec3 v_normal;
    in vec3 v_frag_pos;
    in vec2 v_uv;
    
    uniform sampler2D u_albedo;
    uniform sampler2D u_metallic;
    uniform sampler2D u_roughness;
    uniform sampler2D u_ao;
    
    uniform vec3 light_pos;
    uniform vec3 view_pos;
    
    out vec4 f_color;
    
    const float PI = 3.14159265359;
    
    vec3 fresnelSchlick(float cosTheta, vec3 F0) {
        return F0 + (1.0 - F0) * pow(1.0 - cosTheta, 5.0);
    }
    
    void main() {
        vec3 albedo = texture(u_albedo, v_uv).rgb;
        float metallic = texture(u_metallic, v_uv).r;
        float roughness = texture(u_roughness, v_uv).r;
        float ao = texture(u_ao, v_uv).r;
        
        vec3 N = normalize(v_normal);
        vec3 V = normalize(view_pos - v_frag_pos);
        vec3 L = normalize(light_pos - v_frag_pos);
        vec3 H = normalize(V + L);
        
        // Simplified PBR
        vec3 F0 = mix(vec3(0.04), albedo, metallic);
        vec3 F = fresnelSchlick(max(dot(H, V), 0.0), F0);
        
        float NdotL = max(dot(N, L), 0.0);
        vec3 radiance = vec3(1.0) * NdotL;
        
        vec3 color = (albedo / PI) * radiance * ao;
        f_color = vec4(color, 1.0);
    }
'''

## 5. Environment Maps (Reflections)

Metallic surfaces reflect the environment. We use **cubemaps** for this.

In [ ]:
# Load 6 faces of a cubemap
faces = ['right.jpg', 'left.jpg', 'top.jpg', 'bottom.jpg', 'front.jpg', 'back.jpg']
cubemap_data = [Image.open(f).tobytes() for f in faces]

# Create cubemap texture
cubemap = ctx.texture_cube((512, 512), 3, b''.join(cubemap_data))

# In fragment shader:
'''
uniform samplerCube u_env_map;
vec3 reflection = reflect(-V, N);
vec3 env_color = texture(u_env_map, reflection).rgb;
'''

## 🛠️ Challenge: The Material Showcase

Create a scene with 4 different PBR materials:

1. **Rusty Metal** (high roughness, high metallic)
2. **Polished Chrome** (low roughness, high metallic)
3. **Painted Wood** (medium roughness, low metallic)
4. **Rubber** (high roughness, low metallic)

Download free PBR textures from:
- [CC0 Textures](https://cc0textures.com)
- [Poly Haven](https://polyhaven.com/textures)

If your materials look like photographs, you've mastered PBR. 📷